In [ ]:
# by Thanaporn Patikorn (https://github.com/tpatikorn)
# all right reserved
# This is a part of the maze solving problem used for my AI class
# This file contains the simple Maze class and Maze generation
# Each cell in Maze is either a wall (WALL == 0) or a passage (PASSAGE == 1)

import math
import random
import string

WALL = 0
PASSAGE = 1

DEFAULT_N_ROW = 8
DEFAULT_N_COL = 15
DEFAULT_MIN_DIST = 4
DEFAULT_MAX_RAND_ATTEMPTS = 100
DEFAULT_START_GOAL_COUNT = 1
DEFAULT_LOOP_CHANCE = 0.0


# all error related to Maze objects will be of MazeError type
class MazeError(Exception):
    pass


# the Maze object
class Maze:
    def __init__(self, n_row, n_col, start_goal_count=DEFAULT_START_GOAL_COUNT, min_dist=DEFAULT_MIN_DIST,
                 loop_chance=DEFAULT_LOOP_CHANCE):
        self.__n_row = n_row
        self.__n_col = n_col
        self.__check_adj = []
        self.__maze = [[WALL] * self.__n_col for _ in range(0, self.__n_row)]
        self.__start_goal_count = start_goal_count
        self.__min_dist = min_dist
        self.__loop_chance = loop_chance
        self.__generate_maze()

    # get the dimensions (num rows, num columns) of the maze
    def get_dim(self):
        return self.__n_row, self.__n_col

    # check whether the cell is in the maze or not
    def validate_cell(self, cell):
        if cell[0] >= self.__n_row or cell[0] < 0:
            raise MazeError("the input row" + str(cell[0]) + " is outside of the Maze" + str(self.get_dim()))
        if cell[1] >= self.__n_col or cell[1] < 0:
            raise MazeError("the input col" + str(cell[1]) + " is outside of the Maze" + str(self.get_dim()))

    # return a list of tuple of starting cells and goal cells
    # if i is None or unspecified, the list will contain all the starting cells and goal cells
    # if i is an integer, the *list* containing only the tuple on that index is returned
    def get_start_goal_cells(self, i=None):
        if i is None:
            return self.__start_goal_pairs.copy()
        elif 0 <= i < len(self.__start_goal_pairs):
            return [self.__start_goal_pairs[i]]
        else:
            raise MazeError("Index out of range: there are only",
                            len(self.__start_goal_pairs), "pairs of start-goal pairs. Given i:", i)

    # the same as get_start_goal_cells except only the starting cells are returned
    def get_start_cells(self, i=None):
        return list(map(lambda p: p[0], self.get_start_goal_cells(i)))

    # the same as get_start_goal_cells except only the goal cells are returned
    def get_goal_cells(self, i=None):
        return list(map(lambda p: p[1], self.get_start_goal_cells(i)))

    # return the type of the specified cell (e.g. wall, empty cell) see the constants for details
    # private to hinder students from accessing the cells directly
    def __get_cell(self, cell):
        return self.__maze[cell[0]][cell[1]]

    # set the type of the specified cell (e.g. wall, empty cell) see the constants for details
    # private to hinder students from accessing the cells directly
    def __set_cell(self, cell, value):
        self.__maze[cell[0]][cell[1]] = value

    # return the list of cells adjacent to the specified cell regardless of its type
    # private to hinder students from accessing the cells directly
    def __get_adj(self, cell):
        self.validate_cell(cell)
        adj_cells = []
        row = cell[0]
        col = cell[1]
        if row > 0:
            adj_cells.append((row - 1, col))
        if col < self.__n_col - 1:
            adj_cells.append((row, col + 1))
        if row < self.__n_row - 1:
            adj_cells.append((row + 1, col))
        if col > 0:
            adj_cells.append((row, col - 1))
        return adj_cells

    # return the euclidean distance between the cells using its coordinate
    # private to hinder students from accessing the cells directly
    def __euclidean_distance(self, cell1, cell2):
        self.validate_cell(cell1)
        self.validate_cell(cell2)
        return math.sqrt((cell1[0] - cell2[0]) ** 2 + (cell1[1] - cell2[1]) ** 2)

    # generate maze using Prim's algorithm
    def __generate_maze(self):
        walls = []
        passages = []
        current_cell = (0, 0)
        passages.append(current_cell)
        walls.extend(self.__get_adj(current_cell))
        self.__set_cell(current_cell, PASSAGE)
        while len(walls) > 0:
            current_cell = random.sample(walls, 1)[0]
            adj_cells = self.__get_adj(current_cell)
            if sum(map(lambda c: self.__get_cell(c), adj_cells)) == 1:
                self.__set_cell(current_cell, PASSAGE)
                unvisited = list(filter(lambda c: self.__get_cell(c) != PASSAGE, adj_cells))
                walls.extend(unvisited)
                passages.append(current_cell)
            elif random.random() < self.__loop_chance:
                self.__set_cell(current_cell, PASSAGE)
                unvisited = list(filter(lambda c: self.__get_cell(c) != PASSAGE, adj_cells))
                walls.extend(unvisited)
                passages.append(current_cell)
            walls.remove(current_cell)

        attempts = 0
        start_goal_pairs = []
        while len(start_goal_pairs) < self.__start_goal_count:
            this_pair = random.sample(passages, 2)
            if self.__euclidean_distance(this_pair[0], this_pair[1]) >= self.__min_dist:
                start_goal_pairs.append(this_pair)
            attempts = attempts + 1
            if attempts > DEFAULT_MAX_RAND_ATTEMPTS:
                raise MazeError("The maze is too small for this many start-goal pairs. dim:", self.get_dim(),
                                "number of start-goal pairs:", self.__start_goal_count)

        self.__start_goal_pairs = start_goal_pairs

    # return the list of all empty cells adjacent to the given cell
    # keep the record of which cells were accessed using get_adj()
    def get_adj(self, cell):
        self.__check_adj.append(cell)
        return list(filter(lambda c: self.__get_cell(c) != WALL, self.__get_adj(cell)))

    # return True if cell_to is adjacent to cell_from
    def is_adj(self, cell_from, cell_to):
        return cell_to in self.get_adj(cell_from)

    # return the list of cell accessed using get_adj() in the order of which they were accessed
    def get_check_history(self):
        return self.__check_adj.copy()

    # print the maze in ascii
    def print_maze(self, excel_mode=False):
        if excel_mode:
            print("  | " + " ".join(string.ascii_uppercase[0:self.__n_col]) + " |")
        else:
            print("  | " + " ".join(str(math.floor(i / 10)) for i in range(0, self.__n_col)) + " |")
            print("  | " + " ".join(str(i % 10) for i in range(0, self.__n_col)) + " |")
        print("  " + ('- ' * (self.__n_col + 2)))
        for i in range(0, self.__n_row):
            row_str = "{:02d}".format(i) + '| '
            for j in range(0, self.__n_col):
                current_cell = self.__get_cell((i, j))
                if current_cell == WALL:
                    row_str = row_str + 'X '
                elif current_cell == PASSAGE:
                    row_str = row_str + '  '
                else:
                    raise MazeError("Unrecognizable cell type: ", current_cell)
            row_str = row_str + '|'
            print(row_str)
        print("  " + ('- ' * (self.__n_col + 2)))
        if excel_mode:
            print("  | " + " ".join(string.ascii_uppercase[0:self.__n_col]) + " |")
        else:
            print("  | " + " ".join(str(math.floor(i / 10)) for i in range(0, self.__n_col)) + " |")
            print("  | " + " ".join(str(i % 10) for i in range(0, self.__n_col)) + " |")

    def print_stat(self):
        print("maze dimension:", self.get_dim())
        print("maze starting & goal cells:", *self.get_start_goal_cells(), sep="\n")
        print("maze starting cell:", self.get_start_cells())
        print("maze goal cell:", self.get_goal_cells())


def remove_duplicates(input_list):
    return list(dict.fromkeys(input_list))

# Maze

# Agent (Abstract)

In [9]:
class AgentError(Exception):
    pass


class Agent:
    def __init__(self, this_maze, start_cell, goal_cell):
        self.maze = this_maze
        self.maze.validate_cell(start_cell)
        self.maze.validate_cell(goal_cell)
        self.__start_cell = start_cell
        self.__goal_cell = goal_cell
        self.__visited = set()
        self.__can_visit = {start_cell}
        self.__history = [start_cell]
        self.__reach_goal = False
        self.__backtrack = {start_cell: None}
        self.__correct_path = []

    # return visited cells
    def get_visited_cells(self):
        return self.__visited.copy()

    # return cells that can be visited
    def get_can_visit_cells(self, not_visited_only=False):
        if not_visited_only:
            return set(self.__can_visit) - self.__visited
        else:
            return self.__can_visit.copy()

    # return the starting cell of the Agent
    def get_start_cell(self):
        return self.__start_cell

    # return the goal cell of the Agent
    def get_goal_cell(self):
        return self.__goal_cell

    # move the Agent to the target cell
    # if the target cell is explored (i.e. adjacent to any visited cells),
    # visit that cell, explore the adjacent cells, the target cell is added to the history
    # then return the recently explored adjacent cells
    # otherwise throws an error
    def travel(self, target_cell):
        if self.__reach_goal:
            print("Agent is already at the goal cell.", self.__goal_cell)
        if target_cell in self.__can_visit:
            self.__history.append(target_cell)
            new_adj = set(self.maze.get_adj(target_cell)) - self.__visited
            self.__can_visit = self.__can_visit.union(new_adj)
            self.__visited.add(target_cell)
            print("path",self.__correct_path)
            print("history",self.__history)
            print ("backtrack",self.__backtrack) 
            for next_cell in new_adj:
                self.__backtrack[next_cell] = target_cell
            if target_cell == self.__goal_cell:
                self.__reach_goal = True
                current_cell = self.get_goal_cell()
                while current_cell is not None:
                    self.__correct_path.append(current_cell)
                    current_cell = self.__backtrack[current_cell]

    
            return new_adj
        else:
            raise AgentError("Target cell has not been explored yet: explored cells:",
                             self.__can_visit, ". Target cell:", target_cell)

    # print the history so far of the Agent
    def print_history(self):
        print(*self.__history, sep="-")

    # return whether the Agent is at the goal cell
    def is_at_goal(self):
        return self.__reach_goal

    # use search algorithm to move the Agent from the start cell to the goal cell
    def search(self):
        raise NotImplementedError("This agent has not been implemented!")

    def print_stat(self):
        print("=============================================")
        print("start cell: {}. goal cell: {}. reached goal: {}".format(self.__start_cell, self.__goal_cell,
                                                                       self.__reach_goal))
        if self.__reach_goal:
            print("path to goal (len: {}): {}".format(len(self.__correct_path), self.__correct_path))
            print("steps (len: {}): {}".format(len(self.__history), self.__history))
        print("=============================================")


# RandomAgent (Example of an implementation of Agent)

In [12]:
class RandomAgent(Agent):
    backtrack = {}
    correct_path = []

    def search(self, verbose=False):
        visited = set()  # store all visited cells
        attempts = 1000
        mul =1
        dept = 2
        to_explore = [self.get_start_cell()]  # cells that can be explored
        self.backtrack = {self.get_start_cell(): None}  # a map that keeps the direction back to the start
        while attempts > 0 and not self.is_at_goal():
          mul *=2 #incease when cannot reach goal
          dept =dept+mul #increase deep
          while dept > 0 and not self.is_at_goal():
            print("attemps",attempts)
            print("dept",dept) #remaining deep
            print("mul",mul) #deep increase when cannot reach goal
    
            selected = -1  # selected a last index
            if verbose:
                print("to_explore", to_explore)
                print("selected", selected)
            this_cell = to_explore.pop(selected)  # select the cell
            if verbose:
                print("this_cell", this_cell)
            visited.add(this_cell)  # add the selected cell to visited
            new_adj = set(self.travel(this_cell)) - visited  # new possible cells to explore
            to_explore = remove_duplicates(to_explore + list(new_adj))  # add new cells to to_explore
            for next_cell in new_adj:
                self.backtrack[next_cell] = this_cell  # set backtracks for all cells
            attempts = attempts - 1
            dept = dept -1
            
        if self.is_at_goal():
            current_cell = self.get_goal_cell()  # start the backtracking from the goal back to the start
            while current_cell is not None:
                self.correct_path.append(current_cell)
                current_cell = self.backtrack[current_cell]
            return self.correct_path  # return correct path
        
        raise AgentError("Path is not found")


# Run

In [13]:
random.seed(164428222008 % 1000) # lock seed. use student id
default_maze = Maze(n_row=DEFAULT_N_ROW, n_col=DEFAULT_N_COL,
                    start_goal_count=10, min_dist=DEFAULT_MIN_DIST, 
                    loop_chance=0.15)
default_maze.print_maze()
default_maze.print_stat()
start_goal_cells = default_maze.get_start_goal_cells()
for start, goal in start_goal_cells:
    new_agent = RandomAgent(this_maze=default_maze, start_cell=start, goal_cell=goal)
    new_agent.search(verbose=True)
    new_agent.print_stat()


  | 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 |
  | 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 |
  - - - - - - - - - - - - - - - - - 
00|           X             X     |
01|   X                     X     |
02|   X         X     X     X     |
03|   X   X           X       X   |
04|         X         X       X   |
05| X   X     X   X               |
06|           X                   |
07| X   X   X       X   X   X     |
  - - - - - - - - - - - - - - - - - 
  | 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 |
  | 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 |
maze dimension: (8, 15)
maze starting & goal cells:
[(1, 11), (6, 13)]
[(0, 9), (5, 12)]
[(7, 11), (4, 14)]
[(4, 10), (6, 4)]
[(4, 5), (0, 8)]
[(5, 14), (6, 10)]
[(4, 3), (5, 10)]
[(6, 7), (6, 13)]
[(0, 4), (6, 4)]
[(6, 10), (1, 0)]
maze starting cell: [(1, 11), (0, 9), (7, 11), (4, 10), (4, 5), (5, 14), (4, 3), (6, 7), (0, 4), (6, 10)]
maze goal cell: [(6, 13), (5, 12), (4, 14), (6, 4), (0, 8), (6, 10), (5, 10), (6, 13), (6, 4), (1, 0)]
attemps 1000
dept 4
mul 2
to_explore [(1, 11)]
selec